## 0.0 All Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import joblib
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import pyarabic.araby as araby
import arabicstopwords.arabicstopwords as stp
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import ArabicStemmer
import qalsadi.lemmatizer

# Initialize stemmer and lemmatizer
st = ArabicStemmer()
lemmer = qalsadi.lemmatizer.Lemmatizer()

# Set display options for pandas
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', None)

# Set seaborn style
sns.set()

[nltk_data] Downloading package stopwords to /home/fox/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Load the CSV file 
df = pd.read_csv("TrainingData/Arabic-Original.csv", header=None, names=['text'])

# Display the first five rows of the dataframe
df.head()

,text
0,1|1|بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ
1,1|2|الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
2,1|3|الرَّحْمَٰنِ الرَّحِيمِ
3,1|4|مَالِكِ يَوْمِ الدِّينِ
4,1|5|إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ


In [3]:
# Split the 'text' column on '|' and expand it into separate columns
df = df['text'].str.split('|', expand=True)

# Rename the columns to more descriptive names
df.columns = ['surah_num', 'ayah_num', 'ayah_txt']

# Convert 'surah_num' and 'ayah_num' to integers
df['surah_num'] = df['surah_num'].astype('int')
df['ayah_num'] = df['ayah_num'].astype('int')

# Display the first 10 rows of the dataframe
df.head(10)

,surah_num,ayah_num,ayah_txt
0,1,1,بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ
1,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
2,1,3,الرَّحْمَٰنِ الرَّحِيمِ
3,1,4,مَالِكِ يَوْمِ الدِّينِ
4,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
5,1,6,اهْدِنَا الصِّرَاطَ الْمُسْتَقِيمَ
6,1,7,صِرَاطَ الَّذِينَ أَنْعَمْتَ عَلَيْهِمْ غَيْرِ الْمَغْضُوبِ عَلَيْهِمْ وَلَا الضَّالِّينَ
7,2,1,بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ الم
8,2,2,ذَٰلِكَ الْكِتَابُ لَا رَيْبَ ۛ فِيهِ ۛ هُدًى لِلْمُتَّقِينَ
9,2,3,الَّذِينَ يُؤْمِنُونَ بِالْغَيْبِ وَيُقِيمُونَ الصَّلَاةَ وَمِمَّا رَزَقْنَاهُمْ يُنْفِقُونَ


## 0.1 Text Cleaning

In [4]:
def normalize_chars(txt):
    # Normalize Arabic characters to simplify text processing
    txt = re.sub("[إأٱآا]", "ا", txt)
    txt = re.sub("ى", "ي", txt)
    txt = re.sub("ة", "ه", txt)
    return txt

In [5]:
stopwordlist = set(list(stp.stopwords_list()) + stopwords.words('arabic'))
stopwordlist = [normalize_chars(word) for word in stopwordlist]

In [6]:
def clean_txt(txt):
    # remove tashkeel & tatweel
    txt = araby.strip_diacritics(txt)
    txt = araby.strip_tatweel(txt)
    # normalize chars
    txt = normalize_chars(txt)
    # remove stopwords & punctuation
    txt = ' '.join([token.translate(str.maketrans('','',string.punctuation)) for token in txt.split(' ') if token not in stopwordlist])
    # lemmatizer
    txt_lemmatized = ' '.join([lemmer.lemmatize(token) for token in txt.split(' ')])
    return txt + " " + txt_lemmatized

In [7]:
df['clean_txt'] = df['ayah_txt'].apply(lambda x: clean_txt(x))
df.sample(3)

,surah_num,ayah_num,ayah_txt,clean_txt
2522,21,40,بَلْ تَأْتِيهِمْ بَغْتَةً فَتَبْهَتُهُمْ فَلَا يَسْتَطِيعُونَ رَدَّهَا وَلَا هُمْ يُنْظَرُونَ,تاتيهم فتبهتهم يستطيعون ردها ينظرون تاتيهم بهت استطاع رد نظر
1928,16,28,الَّذِينَ تَتَوَفَّاهُمُ الْمَلَائِكَةُ ظَالِمِي أَنْفُسِهِمْ ۖ فَأَلْقَوُا السَّلَمَ مَا كُنَّا نَعْمَلُ مِنْ سُوءٍ ۚ بَلَىٰ إِنَّ اللَّهَ عَلِيمٌ بِمَا كُنْتُمْ تَعْمَلُونَ,تتوفاهم الملائكه ظالمي فالقوا السلم كنا نعمل سوء الله عليم كنتم تعملون توفى الملائكه ظالم لقا سلم كن عمل سوء الله عليم كنتم عمل
4830,53,47,وَأَنَّ عَلَيْهِ النَّشْأَةَ الْأُخْرَىٰ,النشاه الاخري النشاه الاخري


In [8]:
# get all surah names
def get_surah_names():
    surah_names = []
    url = "https://surahquran.com/quran-search/quran.html"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    all_table = soup.find_all('table')[1]
    for element in all_table.find_all("a"):
        surah_names.append(element.text)
    return surah_names

In [9]:
quran_surah_names = get_surah_names()
quran_surah_names

['الفاتحة',
 'البقرة',
 'آل عمران',
 'النساء',
 'المائدة',
 'الأنعام',
 'الأعراف',
 'الأنفال',
 'التوبة',
 'يونس',
 'هود',
 'يوسف',
 'الرعد',
 'إبراهيم',
 'الحجر',
 'النحل',
 'الإسراء',
 'الكهف',
 'مريم',
 'طه',
 'الأنبياء',
 'الحج',
 'المؤمنون',
 'النور',
 'الفرقان',
 'الشعراء',
 'النمل',
 'القصص',
 'العنكبوت',
 'الروم',
 'لقمان',
 'السجدة',
 'الأحزاب',
 'سبأ',
 'فاطر',
 'يس',
 'الصافات',
 'ص',
 'الزمر',
 'غافر',
 'فصلت',
 'الشورى',
 'الزخرف',
 'الدخان',
 'الجاثية',
 'الأحقاف',
 'محمد',
 'الفتح',
 'الحجرات',
 'ق',
 'الذاريات',
 'الطور',
 'النجم',
 'القمر',
 'الرحمن',
 'الواقعة',
 'الحديد',
 'المجادلة',
 'الحشر',
 'الممتحنة',
 'الصف',
 'الجمعة',
 'المنافقون',
 'التغابن',
 'الطلاق',
 'التحريم',
 'الملك',
 'القلم',
 'الحاقة',
 'المعارج',
 'نوح',
 'الجن',
 'المزّمِّل',
 'المدّثر',
 'القيامة',
 'الإنسان',
 'المرسلات',
 'النبأ',
 'النازعات',
 'عبس',
 'التكوير',
 'الإنفطار',
 'المطففين',
 'الانشقاق',
 'البروج',
 'الطارق',
 'الأعلى',
 'الغاشية',
 'الفجر',
 'البلد',
 'الشمس',
 'الليل',
 'الضحى

In [10]:
# updating df by add Surah name to df
df['surah_name'] = df['surah_num'].apply(lambda x: quran_surah_names[x-1])
df.head(5)

,surah_num,ayah_num,ayah_txt,clean_txt,surah_name
0,1,1,بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ,بسم الله الرحمن الرحيم سم الله رحمن رحيم,الفاتحة
1,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,الحمد لله العالمين حمد له عالم,الفاتحة
2,1,3,الرَّحْمَٰنِ الرَّحِيمِ,الرحمن الرحيم رحمن رحيم,الفاتحة
3,1,4,مَالِكِ يَوْمِ الدِّينِ,مالك يوم الدين مال يوم دين,الفاتحة
4,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,نعبد نستعين عبد استعان,الفاتحة


In [11]:
# grouping the 114 surah

grouped_surah = {'surah_num': [], 'surah_name': [], 'surah_txt': []}

for i in range(1, 115):
    surah_txt = ' '.join(df[df['surah_num'] == i]['ayah_txt'].tolist())
    surah_name = quran_surah_names[i - 1]
    
    grouped_surah['surah_num'].append(i)
    grouped_surah['surah_name'].append(surah_name)
    grouped_surah['surah_txt'].append(surah_txt)

df_surah = pd.DataFrame(grouped_surah)
df_surah.head(1)

,surah_num,surah_name,surah_txt
0,1,الفاتحة,بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ الرَّحْمَٰنِ الرَّحِيمِ مَالِكِ يَوْمِ الدِّينِ إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ اهْدِنَا الصِّرَاطَ الْمُسْتَقِيمَ صِرَاطَ الَّذِينَ أَنْعَمْتَ عَلَيْهِمْ غَيْرِ الْمَغْضُوبِ عَلَيْهِمْ وَلَا الضَّالِّينَ


In [12]:
df.to_csv('df_surah.csv', index=False)
df_surah.to_csv('processed_df_surah.csv', index=False)

## 1.0 TFIDF Training

In [13]:
# Extract the clean text from the dataframe
corpus = df['clean_txt']

# Instantiate the vectorizer object
vectorizer = TfidfVectorizer(ngram_range=(1, 2))

# Vectorize the corpus
corpus_vectorized = vectorizer.fit_transform(corpus)

# save the model to a file using joblib
joblib.dump(vectorizer, 'model.pkl')

# Print the shape of the vectorized corpus
print(f"Vectorized corpus shape: {corpus_vectorized.shape}")

Vectorized corpus shape: (6236, 75140)


In [14]:
print(vectorizer.get_feature_names_out()[:30])

['آتى' 'آتى أجزى' 'آتى اتوا' 'آتى اجرا' 'آتى اجرها' 'آتى اجرهم'
 'آتى اجوركم' 'آتى اجورهم' 'آتى احدا' 'آتى الاخره' 'آتى الحكمه'
 'آتى الزكاه' 'آتى الله' 'آتى اوتي' 'آتى اوتيتم' 'آتى باذن' 'آتى تب'
 'آتى حذر' 'آتى خير' 'آتى ردى' 'آتى سفهاء' 'آتى شاء' 'آتى صحف' 'آتى فضل'
 'آتى فقراء' 'آتى فل' 'آتى قرب' 'آتى مال' 'آتى ملك' 'آتى موثق']


In [15]:
def show_best_results(df_quran, scores_array, top_n=20):
   
    # Sort the scores in descending order and get the corresponding indices
    sorted_indices = scores_array.argsort()[::-1]

    # Iterate over the top n results and print them to the console
    for position, idx in enumerate(sorted_indices[:top_n]):
        row = df_quran.iloc[idx]
        ayah = row['ayah_txt']
        ayah_num = row['ayah_num']
        surah_name = row['surah_name']
        score = scores_array[idx]

        # Only display ayahs with non-zero scores
        if score > 0:
            print(ayah)
            # print(score)
            print(f"أية رقم {ayah_num} من سورة {surah_name}")
            print("====================================")

In [16]:
def run_arabic_search_engine(query):
    
    # Clean the query text and vectorize it
    query_cleaned = clean_txt(query)
    query_vectorized = vectorizer.transform([query_cleaned])
    
    # Compute the scores for each ayah in the corpus
    scores = query_vectorized.dot(corpus_vectorized.transpose())
    scores_array = scores.toarray()[0]
    
    # Display the top matching ayahs to the console
    show_best_results(df, scores_array)

In [17]:
query = "محمد رسول الله"
run_arabic_search_engine(query)

مَا كَانَ مُحَمَّدٌ أَبَا أَحَدٍ مِنْ رِجَالِكُمْ وَلَٰكِنْ رَسُولَ اللَّهِ وَخَاتَمَ النَّبِيِّينَ ۗ وَكَانَ اللَّهُ بِكُلِّ شَيْءٍ عَلِيمًا
أية رقم 40 من سورة الأحزاب
وَمَا مُحَمَّدٌ إِلَّا رَسُولٌ قَدْ خَلَتْ مِنْ قَبْلِهِ الرُّسُلُ ۚ أَفَإِنْ مَاتَ أَوْ قُتِلَ انْقَلَبْتُمْ عَلَىٰ أَعْقَابِكُمْ ۚ وَمَنْ يَنْقَلِبْ عَلَىٰ عَقِبَيْهِ فَلَنْ يَضُرَّ اللَّهَ شَيْئًا ۗ وَسَيَجْزِي اللَّهُ الشَّاكِرِينَ
أية رقم 144 من سورة آل عمران
مُحَمَّدٌ رَسُولُ اللَّهِ ۚ وَالَّذِينَ مَعَهُ أَشِدَّاءُ عَلَى الْكُفَّارِ رُحَمَاءُ بَيْنَهُمْ ۖ تَرَاهُمْ رُكَّعًا سُجَّدًا يَبْتَغُونَ فَضْلًا مِنَ اللَّهِ وَرِضْوَانًا ۖ سِيمَاهُمْ فِي وُجُوهِهِمْ مِنْ أَثَرِ السُّجُودِ ۚ ذَٰلِكَ مَثَلُهُمْ فِي التَّوْرَاةِ ۚ وَمَثَلُهُمْ فِي الْإِنْجِيلِ كَزَرْعٍ أَخْرَجَ شَطْأَهُ فَآزَرَهُ فَاسْتَغْلَظَ فَاسْتَوَىٰ عَلَىٰ سُوقِهِ يُعْجِبُ الزُّرَّاعَ لِيَغِيظَ بِهِمُ الْكُفَّارَ ۗ وَعَدَ اللَّهُ الَّذِينَ آمَنُوا وَعَمِلُوا الصَّالِحَاتِ مِنْهُمْ مَغْفِرَةً وَأَجْرًا عَظِيمًا
أية رقم 29 من سورة الفتح
رَسُولٌ مِنَ اللَّه

In [18]:
query = "عيسي بن مريم"
run_arabic_search_engine(query)

ذَٰلِكَ عِيسَى ابْنُ مَرْيَمَ ۚ قَوْلَ الْحَقِّ الَّذِي فِيهِ يَمْتَرُونَ
أية رقم 34 من سورة مريم
إِذْ قَالَتِ الْمَلَائِكَةُ يَا مَرْيَمُ إِنَّ اللَّهَ يُبَشِّرُكِ بِكَلِمَةٍ مِنْهُ اسْمُهُ الْمَسِيحُ عِيسَى ابْنُ مَرْيَمَ وَجِيهًا فِي الدُّنْيَا وَالْآخِرَةِ وَمِنَ الْمُقَرَّبِينَ
أية رقم 45 من سورة آل عمران
إِذْ قَالَ الْحَوَارِيُّونَ يَا عِيسَى ابْنَ مَرْيَمَ هَلْ يَسْتَطِيعُ رَبُّكَ أَنْ يُنَزِّلَ عَلَيْنَا مَائِدَةً مِنَ السَّمَاءِ ۖ قَالَ اتَّقُوا اللَّهَ إِنْ كُنْتُمْ مُؤْمِنِينَ
أية رقم 112 من سورة المائدة
يَا أَهْلَ الْكِتَابِ لَا تَغْلُوا فِي دِينِكُمْ وَلَا تَقُولُوا عَلَى اللَّهِ إِلَّا الْحَقَّ ۚ إِنَّمَا الْمَسِيحُ عِيسَى ابْنُ مَرْيَمَ رَسُولُ اللَّهِ وَكَلِمَتُهُ أَلْقَاهَا إِلَىٰ مَرْيَمَ وَرُوحٌ مِنْهُ ۖ فَآمِنُوا بِاللَّهِ وَرُسُلِهِ ۖ وَلَا تَقُولُوا ثَلَاثَةٌ ۚ انْتَهُوا خَيْرًا لَكُمْ ۚ إِنَّمَا اللَّهُ إِلَٰهٌ وَاحِدٌ ۖ سُبْحَانَهُ أَنْ يَكُونَ لَهُ وَلَدٌ ۘ لَهُ مَا فِي السَّمَاوَاتِ وَمَا فِي الْأَرْضِ ۗ وَكَفَىٰ بِاللَّهِ وَكِيلًا
أية رقم 171 من سورة النساء
ق

In [19]:
query = 'الربا'
run_arabic_search_engine(query)

الَّذِينَ يَأْكُلُونَ الرِّبَا لَا يَقُومُونَ إِلَّا كَمَا يَقُومُ الَّذِي يَتَخَبَّطُهُ الشَّيْطَانُ مِنَ الْمَسِّ ۚ ذَٰلِكَ بِأَنَّهُمْ قَالُوا إِنَّمَا الْبَيْعُ مِثْلُ الرِّبَا ۗ وَأَحَلَّ اللَّهُ الْبَيْعَ وَحَرَّمَ الرِّبَا ۚ فَمَنْ جَاءَهُ مَوْعِظَةٌ مِنْ رَبِّهِ فَانْتَهَىٰ فَلَهُ مَا سَلَفَ وَأَمْرُهُ إِلَى اللَّهِ ۖ وَمَنْ عَادَ فَأُولَٰئِكَ أَصْحَابُ النَّارِ ۖ هُمْ فِيهَا خَالِدُونَ
أية رقم 275 من سورة البقرة
يَا أَيُّهَا الَّذِينَ آمَنُوا اتَّقُوا اللَّهَ وَذَرُوا مَا بَقِيَ مِنَ الرِّبَا إِنْ كُنْتُمْ مُؤْمِنِينَ
أية رقم 278 من سورة البقرة
يَا أَيُّهَا الَّذِينَ آمَنُوا لَا تَأْكُلُوا الرِّبَا أَضْعَافًا مُضَاعَفَةً ۖ وَاتَّقُوا اللَّهَ لَعَلَّكُمْ تُفْلِحُونَ
أية رقم 130 من سورة آل عمران
وَمَا آتَيْتُمْ مِنْ رِبًا لِيَرْبُوَ فِي أَمْوَالِ النَّاسِ فَلَا يَرْبُو عِنْدَ اللَّهِ ۖ وَمَا آتَيْتُمْ مِنْ زَكَاةٍ تُرِيدُونَ وَجْهَ اللَّهِ فَأُولَٰئِكَ هُمُ الْمُضْعِفُونَ
أية رقم 39 من سورة الروم
يَمْحَقُ اللَّهُ الرِّبَا وَيُرْبِي الصَّدَقَاتِ ۗ وَاللَّهُ لَا يُحِبُّ كُلَّ كَفّ

In [20]:
query = 'الجنة و النار'
run_arabic_search_engine(query)

لَا يَسْتَوِي أَصْحَابُ النَّارِ وَأَصْحَابُ الْجَنَّةِ ۚ أَصْحَابُ الْجَنَّةِ هُمُ الْفَائِزُونَ
أية رقم 20 من سورة الحشر
مَثَلُ الْجَنَّةِ الَّتِي وُعِدَ الْمُتَّقُونَ ۖ تَجْرِي مِنْ تَحْتِهَا الْأَنْهَارُ ۖ أُكُلُهَا دَائِمٌ وَظِلُّهَا ۚ تِلْكَ عُقْبَى الَّذِينَ اتَّقَوْا ۖ وَعُقْبَى الْكَافِرِينَ النَّارُ
أية رقم 35 من سورة الرعد
مِنَ الْجِنَّةِ وَالنَّاسِ
أية رقم 6 من سورة الناس
وَجَعَلُوا بَيْنَهُ وَبَيْنَ الْجِنَّةِ نَسَبًا ۚ وَلَقَدْ عَلِمَتِ الْجِنَّةُ إِنَّهُمْ لَمُحْضَرُونَ
أية رقم 158 من سورة الصافات
فَإِنَّ الْجَنَّةَ هِيَ الْمَأْوَىٰ
أية رقم 41 من سورة النازعات
وَإِذَا الْجَنَّةُ أُزْلِفَتْ
أية رقم 13 من سورة التكوير
وَيُدْخِلُهُمُ الْجَنَّةَ عَرَّفَهَا لَهُمْ
أية رقم 6 من سورة محمد
النَّارِ ذَاتِ الْوَقُودِ
أية رقم 5 من سورة البروج
وَأُزْلِفَتِ الْجَنَّةُ لِلْمُتَّقِينَ
أية رقم 90 من سورة الشعراء
نَارٌ حَامِيَةٌ
أية رقم 11 من سورة القارعة
هَٰذِهِ النَّارُ الَّتِي كُنْتُمْ بِهَا تُكَذِّبُونَ
أية رقم 14 من سورة الطور
تِلْكَ الْجَنَّةُ الَّتِي نُورِثُ مِنْ عِبَادِنَا مَنْ ك

In [21]:
query = 'الفائزون'
run_arabic_search_engine(query)

إِنِّي جَزَيْتُهُمُ الْيَوْمَ بِمَا صَبَرُوا أَنَّهُمْ هُمُ الْفَائِزُونَ
أية رقم 111 من سورة المؤمنون
وَمَنْ يُطِعِ اللَّهَ وَرَسُولَهُ وَيَخْشَ اللَّهَ وَيَتَّقْهِ فَأُولَٰئِكَ هُمُ الْفَائِزُونَ
أية رقم 52 من سورة النور
لَا يَسْتَوِي أَصْحَابُ النَّارِ وَأَصْحَابُ الْجَنَّةِ ۚ أَصْحَابُ الْجَنَّةِ هُمُ الْفَائِزُونَ
أية رقم 20 من سورة الحشر
الَّذِينَ آمَنُوا وَهَاجَرُوا وَجَاهَدُوا فِي سَبِيلِ اللَّهِ بِأَمْوَالِهِمْ وَأَنْفُسِهِمْ أَعْظَمُ دَرَجَةً عِنْدَ اللَّهِ ۚ وَأُولَٰئِكَ هُمُ الْفَائِزُونَ
أية رقم 20 من سورة التوبة


In [22]:
query = 'يوم القيامة'
run_arabic_search_engine(query)

يَسْأَلُ أَيَّانَ يَوْمُ الْقِيَامَةِ
أية رقم 6 من سورة القيامة
ثُمَّ إِنَّكُمْ يَوْمَ الْقِيَامَةِ تُبْعَثُونَ
أية رقم 16 من سورة المؤمنون
ثُمَّ إِنَّكُمْ يَوْمَ الْقِيَامَةِ عِنْدَ رَبِّكُمْ تَخْتَصِمُونَ
أية رقم 31 من سورة الزمر
خَالِدِينَ فِيهِ ۖ وَسَاءَ لَهُمْ يَوْمَ الْقِيَامَةِ حِمْلًا
أية رقم 101 من سورة طه
إِنَّ رَبَّكَ هُوَ يَفْصِلُ بَيْنَهُمْ يَوْمَ الْقِيَامَةِ فِيمَا كَانُوا فِيهِ يَخْتَلِفُونَ
أية رقم 25 من سورة السجدة
مَنْ أَعْرَضَ عَنْهُ فَإِنَّهُ يَحْمِلُ يَوْمَ الْقِيَامَةِ وِزْرًا
أية رقم 100 من سورة طه
وَكُلُّهُمْ آتِيهِ يَوْمَ الْقِيَامَةِ فَرْدًا
أية رقم 95 من سورة مريم
اللَّهُ يَحْكُمُ بَيْنَكُمْ يَوْمَ الْقِيَامَةِ فِيمَا كُنْتُمْ فِيهِ تَخْتَلِفُونَ
أية رقم 69 من سورة الحج
يُضَاعَفْ لَهُ الْعَذَابُ يَوْمَ الْقِيَامَةِ وَيَخْلُدْ فِيهِ مُهَانًا
أية رقم 69 من سورة الفرقان
أَمْ لَكُمْ أَيْمَانٌ عَلَيْنَا بَالِغَةٌ إِلَىٰ يَوْمِ الْقِيَامَةِ ۙ إِنَّ لَكُمْ لَمَا تَحْكُمُونَ
أية رقم 39 من سورة القلم
إِنَّمَا جُعِلَ السَّبْتُ عَلَى الَّذِينَ اخْتَلَفُوا فِيهِ ۚ وَإِن

In [23]:
query = 'ورحمتي وسعت كل شيء'
run_arabic_search_engine(query)

وَاكْتُبْ لَنَا فِي هَٰذِهِ الدُّنْيَا حَسَنَةً وَفِي الْآخِرَةِ إِنَّا هُدْنَا إِلَيْكَ ۚ قَالَ عَذَابِي أُصِيبُ بِهِ مَنْ أَشَاءُ ۖ وَرَحْمَتِي وَسِعَتْ كُلَّ شَيْءٍ ۚ فَسَأَكْتُبُهَا لِلَّذِينَ يَتَّقُونَ وَيُؤْتُونَ الزَّكَاةَ وَالَّذِينَ هُمْ بِآيَاتِنَا يُؤْمِنُونَ
أية رقم 156 من سورة الأعراف
إِنَّمَا إِلَٰهُكُمُ اللَّهُ الَّذِي لَا إِلَٰهَ إِلَّا هُوَ ۚ وَسِعَ كُلَّ شَيْءٍ عِلْمًا
أية رقم 98 من سورة طه
الَّذِينَ يَحْمِلُونَ الْعَرْشَ وَمَنْ حَوْلَهُ يُسَبِّحُونَ بِحَمْدِ رَبِّهِمْ وَيُؤْمِنُونَ بِهِ وَيَسْتَغْفِرُونَ لِلَّذِينَ آمَنُوا رَبَّنَا وَسِعْتَ كُلَّ شَيْءٍ رَحْمَةً وَعِلْمًا فَاغْفِرْ لِلَّذِينَ تَابُوا وَاتَّبَعُوا سَبِيلَكَ وَقِهِمْ عَذَابَ الْجَحِيمِ
أية رقم 7 من سورة غافر
مِنْ أَيِّ شَيْءٍ خَلَقَهُ
أية رقم 18 من سورة عبس
اللَّهُ خَالِقُ كُلِّ شَيْءٍ ۖ وَهُوَ عَلَىٰ كُلِّ شَيْءٍ وَكِيلٌ
أية رقم 62 من سورة الزمر
ذَٰلِكُمُ اللَّهُ رَبُّكُمْ ۖ لَا إِلَٰهَ إِلَّا هُوَ ۖ خَالِقُ كُلِّ شَيْءٍ فَاعْبُدُوهُ ۚ وَهُوَ عَلَىٰ كُلِّ شَيْءٍ وَكِيلٌ
أية رقم 102 من سورة الأنعام
وَ

In [24]:
query = 'الصلاه'
run_arabic_search_engine(query)

فَإِذَا قَضَيْتُمُ الصَّلَاةَ فَاذْكُرُوا اللَّهَ قِيَامًا وَقُعُودًا وَعَلَىٰ جُنُوبِكُمْ ۚ فَإِذَا اطْمَأْنَنْتُمْ فَأَقِيمُوا الصَّلَاةَ ۚ إِنَّ الصَّلَاةَ كَانَتْ عَلَى الْمُؤْمِنِينَ كِتَابًا مَوْقُوتًا
أية رقم 103 من سورة النساء
اتْلُ مَا أُوحِيَ إِلَيْكَ مِنَ الْكِتَابِ وَأَقِمِ الصَّلَاةَ ۖ إِنَّ الصَّلَاةَ تَنْهَىٰ عَنِ الْفَحْشَاءِ وَالْمُنْكَرِ ۗ وَلَذِكْرُ اللَّهِ أَكْبَرُ ۗ وَاللَّهُ يَعْلَمُ مَا تَصْنَعُونَ
أية رقم 45 من سورة العنكبوت
الَّذِينَ يُقِيمُونَ الصَّلَاةَ وَمِمَّا رَزَقْنَاهُمْ يُنْفِقُونَ
أية رقم 3 من سورة الأنفال
الَّذِينَ يُؤْمِنُونَ بِالْغَيْبِ وَيُقِيمُونَ الصَّلَاةَ وَمِمَّا رَزَقْنَاهُمْ يُنْفِقُونَ
أية رقم 3 من سورة البقرة
وَأَنْ أَقِيمُوا الصَّلَاةَ وَاتَّقُوهُ ۚ وَهُوَ الَّذِي إِلَيْهِ تُحْشَرُونَ
أية رقم 72 من سورة الأنعام
مُنِيبِينَ إِلَيْهِ وَاتَّقُوهُ وَأَقِيمُوا الصَّلَاةَ وَلَا تَكُونُوا مِنَ الْمُشْرِكِينَ
أية رقم 31 من سورة الروم
الَّذِينَ يُقِيمُونَ الصَّلَاةَ وَيُؤْتُونَ الزَّكَاةَ وَهُمْ بِالْآخِرَةِ هُمْ يُوقِنُونَ
أية رقم 3 من سورة النمل


In [25]:
# %%writefile requirements.txt
# Arabic-Stopwords
# qalsadi
# nltk
# pyarabic
# sklearn
# numpy
# string
# re
# joblib